In [1]:
import pandas as pd
import numpy as np

from skopt.space import Real, Integer
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

from scipy.stats import qmc  

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
df = pd.read_csv('Updated_Drone_Data.csv')

df

,Maximum Principal Elastic Strain (m/m),Equivalent Elastic Strain (m/m),Length (mm),Depth (mm)
0,1.660000e-08,2.170000e-08,403.1,101.0
1,1.490000e-08,1.930000e-08,399.9,100.9
2,1.220000e-08,1.690000e-08,403.0,99.5
3,1.100000e-08,1.450000e-08,397.8,99.7
4,9.810000e-09,1.210000e-08,398.0,99.1
...,...,...,...,...
75,-3.580000e-10,1.030000e-09,431.0,108.7
76,4.160000e-09,6.940000e-09,385.0,119.2
77,6.220000e-09,6.070000e-09,410.5,116.3
78,4.400000e-10,7.300000e-09,385.0,97.8


In [49]:
# Define the hyperparameter space
hyp_space  = [
    Integer(3, 15, name='num_layers'), # Num of layers in the network (depth)
    Integer(50, 300, name='num_units'), # Num of neurons in each hidden layer (width)
    Real(0.00001, 0.2, prior='log-uniform', name='learning_rate'), # Steps size at each iteration 
    Real(0.0, 0.7, name='dropout_rate'), # Probability of droping out a neuron
    Integer(20, 200, name='batch_size'), # Num of samples per batch
    Integer(50, 250, name='epochs') # Num of epochs (iterations over the entire dataset) during training
]

# Defining the Neural Network Model
def NN_model(num_layers, num_units, learning_rate, dropout_rate):
    inputs = Input(shape=(3,))
    x = Dense(num_units, activation='relu', kernel_regularizer='l2')(inputs)
    for _ in range(num_layers - 1):
        x = Dense(num_units, activation='relu', kernel_regularizer='l2')(x)
        x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Function for MC Dropout predictions
def mc_dropout_predictions(model, X, num_samples=50):
    predictions = np.zeros((num_samples, X.shape[0]))
    for i in range(num_samples):
        predictions[i, :] = model(X, training=True).numpy().flatten()
    prediction_mean = predictions.mean(axis=0)
    prediction_std = predictions.std(axis=0)
    return prediction_mean, prediction_std

In [50]:
# Define the objective function to minimize
@use_named_args(hyp_space)
def objective(**params):
    num_layers = params['num_layers']
    num_units = params['num_units']
    learning_rate = params['learning_rate']
    dropout_rate = params['dropout_rate']
    batch_size = params['batch_size']
    epochs = params['epochs']

    model = NN_model(num_layers, num_units, learning_rate, dropout_rate)

    # Define K-fold cross-validation
    kfold = KFold(n_splits=3, shuffle=True, random_state=0)
    scores = []

    # Perform cross-validation
    for train_idx, val_idx in kfold.split(df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']]):
        X_train, X_val = df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']].iloc[train_idx], df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']].iloc[val_idx]
        y_train, y_val = df[['Equivalent Elastic Strain (m/m)']].iloc[train_idx], df[['Equivalent Elastic Strain (m/m)']].iloc[val_idx]

        # Standardize the features
        scaler_x = StandardScaler().fit(X_train)
        X_train_scaled = scaler_x.transform(X_train)
        X_val_scaled = scaler_x.transform(X_val)

        # Train the model
        # early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, verbose=0, validation_data=(X_val_scaled, y_val))

        # Evaluate the model
        score = model.evaluate(X_val_scaled, y_val, verbose=0)
        scores.append(score)
    
    return np.mean(scores)

# Perform Bayesian optimization
result = gp_minimize(objective, hyp_space, n_calls=30, random_state=0, acq_func='EI')

# Output best hyperparameters from BayesOpt
print("Best hyperparameters:")
print("num_layers:", result.x[0])
print("num_units:", result.x[1])
print("learning_rate:", result.x[2])
print("dropout_rate:", result.x[3])
print("batch_size:", result.x[4])
print("epochs:", result.x[5])

# Train the model with the best hyperparameters
best_model = NN_model(
    num_layers=result.x[0],
    num_units=result.x[1],
    learning_rate=result.x[2],
    dropout_rate=result.x[3],
)

In [ ]:
# Standardize the initial dataset
scaler_x = StandardScaler().fit(df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']])
X_scaled = scaler_x.transform(df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']])

# Train the best model on the initial dataset
best_model.fit(X_scaled, df[['Equivalent Elastic Strain (m/m)']], epochs=result.x[5], batch_size=result.x[4], verbose=1)

# # Produce Meshgrid of results with C.I. 
# x1_range = np.linspace(360, 470, 100)
# x2_range  = np.linspace(360, 470, 100)

# x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)
# x_grid = np.c_[x1_grid.ravel(), x2_grid.ravel()]

# x_grid = scaler_x.transform(x_grid)

# # Perform MC Dropout predictions for entire meshgrid
# pred_mean, pred_std = mc_dropout_predictions(best_model, x_grid)

# # Define range for inputs
# bounds = np.array([[360, 470], [360, 470]])

# # Identify top 10 points with highest uncertainty
# num_new_points = 10

# sampler = qmc.LatinHypercube(d=2)  # LHS in a 2D space
# lhs_sample = sampler.random(n=1000)
# lhs_points = qmc.scale(lhs_sample, bounds[:, 0], bounds[:, 1])

# lhs_points_standardized = scaler_x.transform(lhs_points)

# lhs_mean, lhs_std = mc_dropout_predictions(best_model, lhs_points_standardized)

# percentile_threshold = 90 
# threshold_value = np.percentile(lhs_mean, percentile_threshold)

# # Filter points above the threshold
# points_above_threshold = lhs_points[lhs_mean >= threshold_value]

# num_new_points = 10
# if len(points_above_threshold) > num_new_points:
#     selected_indices = np.random.choice(len(points_above_threshold), num_new_points, replace=False)
#     selected_points = points_above_threshold[selected_indices]
# else:
#     selected_points = points_above_threshold

# new_points_df = pd.DataFrame(selected_points, columns=['x','y'])

# # Ploting mean predictions and confidence intervals
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# # Reshaping predictions back to grid shape
# pred_mean_grid = pred_mean.reshape(x1_grid.shape)
# pred_upper_grid = (pred_mean + (1.96 * pred_std)).reshape(x1_grid.shape)
# pred_lower_grid = (pred_mean - (1.96 * pred_std)).reshape(x1_grid.shape)

# mean_surface = ax.plot_surface(x1_grid, x2_grid, pred_mean_grid, color='blue', alpha=0.5, label='Mean Prediction')
# upper_surface = ax.plot_surface(x1_grid, x2_grid, pred_upper_grid, color='red', alpha=0.3, label='Upper Bound (95% CI)')
# lower_surface = ax.plot_surface(x1_grid, x2_grid, pred_lower_grid, color='green', alpha=0.3, label='Lower Bound (95% CI)')

# # Plot next points for sampling
# ax.scatter(selected_points[:, 0], selected_points[:, 1], color='black', marker='o', label='Next Sampling Points')

# # Plot Labels
# ax.set_title('Neural Network Predictions with Confidence Intervals')
# ax.set_xlabel('Length (mm)')
# ax.set_ylabel('Breadth (mm)')
# ax.set_zlabel('Equivalent Elastic Strain (m/m)')

# # Legend
# legend_elements = [mean_surface, upper_surface, lower_surface]
# fig.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(0.1, 0.9))

# # Show the plot
# plt.show()

Epoch 1/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 30.0232
Epoch 2/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 27.2649
Epoch 3/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 24.6844
Epoch 4/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 22.2783
Epoch 5/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 20.0444
Epoch 6/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 17.9760
Epoch 7/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 16.0676
Epoch 8/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 14.3132
Epoch 9/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 12.7059
Epoch 10/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 11.2383
Epoch 11/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 9.9034
Epoch 12/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 8.6938
Epoch 13/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 7.6019
Epoch 14/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 6.6203
Epoch 15/237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 5.7415
Epoch 16/23

In [ ]:
from sklearn.model_selection import train_test_split

train_X = df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']].to_numpy()
train_y = df['Equivalent Elastic Strain (m/m)'].to_numpy()

#Splits data into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y ,test_size=0.20)

# Standardize the features
scaler_x = StandardScaler().fit(X_train)
X_train = scaler_x.transform(X_train)
X_test = scaler_x.transform(X_test) 

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
predictions = best_model.predict(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Mean Squared Error (MSE): 4.7675638789713744e-11
Mean Absolute Error (MAE): 6.904751251182301e-06
R² Score: -973984.7576474203
